In [ ]:
import sys, os, time, cv2
from os import listdir
import numpy as np

timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../.')) 

from makeStableBaselinesILEnv import makeStableBaselinesILEnv

In [ ]:
 # Show files in folder
base_path = os.path.abspath('')
trajRecFolder = os.path.join(base_path, "../../test/data/Discrete/Normal")
#trajRecFolder = os.path.join(base_path, "../../test/data/Discrete/HC")
#trajRecFolder = os.path.join(base_path, "../../test/data/MultiDiscrete/Normal")
#trajRecFolder = os.path.join(base_path, "../../test/data/MultiDiscrete/HC")
trajectoriesFiles = [os.path.join(trajRecFolder, f) for f in listdir(trajRecFolder) if os.path.isfile(os.path.join(trajRecFolder, f))]
print(trajectoriesFiles)

In [ ]:
gameId = "doapp"
diambraILKwargs = {}
diambraILKwargs["trajFilesList"] = trajectoriesFiles
diambraILKwargs["totalCpus"] = 1

hardCore = True if trajRecFolder[-2:] == "HC" else False

In [ ]:
# Additional obs key list
keyToAdd = []
keyToAdd.append("actionsBuf") # env.actBufLen*(env.n_actions[0]+env.n_actions[1])

if gameId != "tektagt":                                                         
    keyToAdd.append("ownHealth")   # 1                                            
    keyToAdd.append("oppHealth")   # 1                                                
else:                                                                           
    keyToAdd.append("ownHealth1")    # 1                                             
    keyToAdd.append("ownHealth2")    # 1                                             
    keyToAdd.append("oppHealth1")    # 1                                              
    keyToAdd.append("oppHealth2")    # 1  
    keyToAdd.append("ownActiveChar") # 1
    keyToAdd.append("oppActiveChar") # 1
    
keyToAdd.append("ownPosition")     # 1
keyToAdd.append("oppPosition")     # 1
keyToAdd.append("stage")           # 1
#keyToAdd.append("ownChar")       # len(env.charNames)
#keyToAdd.append("oppChar")       # len(env.charNames)

In [ ]:
envId = "Test"
env = makeStableBaselinesILEnv(envId, diambraILKwargs, timeDepSeed, hardCore=hardCore, 
                               keyToAdd=keyToAdd, useSubprocess=True)

In [ ]:
observation = env.reset()[0]
env.render(mode="human")

In [ ]:
nChars = env.get_attr("nChars")[0]
charNames = env.get_attr("charNames")[0]
nActions = env.get_attr("nActions")[0]
actBufLen = env.get_attr("actBufLen")[0]
playerSide = env.get_attr("playerSide")[0]

In [ ]:
limAct = [None, None]
for idx in range(2):
    limAct[idx] = [actBufLen*nActions[0], 
                   actBufLen*nActions[0] + actBufLen*nActions[1]]
    
 # Visualize Obs content
def showObs(observation, waitKey):
    
    shp = observation.shape
    additionalPar = int(observation[0,0,shp[2]-1])
        
    # 1P        
    #nScalarAddPar = additionalPar - 2*nChars
    #                - actBufLen*(nActions[0]+nActions[1])
    nScalarAddPar = additionalPar\
                    - actBufLen*(nActions[0]+nActions[1])

    print("Additional Par = ", additionalPar)
    print("N scalar actions = ", nScalarAddPar)
    
    addPar = observation[:,:,shp[2]-1]
    addPar = np.reshape(addPar, (-1))
    addPar = addPar[1:additionalPar+1]
    actions = addPar[0:additionalPar-nScalarAddPar]#-nChars]

    moveActionsP1   = actions[0:limAct[0][0]]
    attackActionsP1 = actions[limAct[0][0]:limAct[0][1]]
    moveActionsP1   = np.reshape(moveActionsP1, (actBufLen,-1))
    attackActionsP1 = np.reshape(attackActionsP1, (actBufLen,-1))
    print("Move actions P1 =\n", moveActionsP1)
    print("Attack actions P1 =\n ", attackActionsP1)
        
    others = addPar[additionalPar-nScalarAddPar:]#-nChars:]
    print("ownHealth = ", others[0])
    print("oppHealth = ", others[1])
    print("ownPosition = ", others[2])
    print("oppPosition = ", others[3])
    print("stage = ", others[4])
    #print("Playing Char  = ", charNames[list(others[nScalarAddPar:
    #                                                nScalarAddPar + nChars]).index(1.0)])
        
    obs = np.array(observation).astype(np.float32)
    
    for idx in range(shp[2]-1):
        cv2.imshow("image"+str(idx), obs[:,:,idx])
    
    cv2.waitKey(waitKey)

In [ ]:
env.env_method("trajSummary")

In [ ]:
showObs(observation, 0)

In [ ]:
cumulativeEpRew = 0.0
cumulativeEpRewAll = []

maxNumEp = 10
currNumEp = 0

procIdx = 0

while currNumEp < maxNumEp:
       
    dummyActions = [0 for i in range(diambraILKwargs["totalCpus"])]
    observation, reward, done, info = env.step(dummyActions)
    
    if np.any(env.get_attr("exhausted")):
        break
    
    env.render(mode="human")
    
    observation = observation[procIdx]
    reward = reward[procIdx]
    done = done[procIdx]
    action = info[procIdx]["action"]
    print("Action = ", action)
    print("Reward = ", reward)
    #if done:
    #    observation = info[procIdx]["terminal_observation"]
    
    # Visualize observations content
    showObs(observation, 0) # Keep space bar pressed to continue env execution
 
    cumulativeEpRew += reward
    
    if done:
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeEpRew = 0.0
        
print("All ep. rewards =", cumulativeEpRewAll)   
print("Mean cumulative reward =", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward =", np.std(cumulativeEpRewAll))       